# Wikipedia

### Wikipedia API

If you intend to do any scraping projects or automated requests, consider alternatives such as Pywikipediabot or MediaWiki API, which has other superior features.

* wikipedia.search('keywords', results=2)
* wikipedia.suggest('keyword')
* wikipedia.summary('keywords', sentences=2)
* wikipedia.page('keywords')
* wikipedia.page('keywords').content
* wikipedia.page('keywords').references
* wikipedia.page('keywords').title
* wikipedia.page('keywords').url
* wikipedia.page('keywords').categories
* wikipedia.page('keywords').content
* wikipedia.page('keywords').links
* wikipedia.geosearch(33.2075, 97.1526)
* wikipedia.set_lang('hi')
* wikipedia.languages()
* wikipedia.page('keywords').images[0]
* wikipedia.page('keywords').html()

In [ ]:
# pip install wikipedia

In [ ]:
# https://kleiber.me/blog/2017/07/22/tutorial-lda-wikipedia/
import pandas as pd
import random
import wikipedia

# rtitles = wikipedia.random(5)

# get 5 Wikipedia page titles based on keywords or manually enter in keywords list
titles = []
keywords = ['Titanic', 'JP Morgan', 'immigration', 'suffrage', 'racist']
for key in keywords:
    title = wikipedia.search(key, results=1)
    titles.append(title[0])

print(titles)
data = []

for title in titles:
    # disambiguous error fix
    try:
        data.append([title, wikipedia.page(title, auto_suggest=False).content, wikipedia.summary(title, auto_suggest=False, sentences=5)])
    except wikipedia.exceptions.DisambiguationError as e:
        s = random.choice(e.options)
        data.append([title, wikipedia.page(s).content,  wikipedia.summary(title, auto_suggest=False, sentences=5)])

df = pd.DataFrame(data, columns=['title', 'content', 'summary'])
df.head()

['Titanic', 'J. P. Morgan', 'Immigration', 'Suffrage', 'Racism']


,title,content,summary
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope..."
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar..."
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc..."
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...


### LDA (Latent Dirichlet Allocation)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(df['summary'].values.astype('U'))

model = LatentDirichletAllocation(n_components=10, random_state=42)
model.fit(vectors)

for index, topic in enumerate(model.components_):
    print(f'Topic {index} top words: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]]}')

Topic 0 top words: ['international', 'people', 'public', 'distinct', 'term']
Topic 1 top words: ['active', 'elections', 'vote', 'right', 'suffrage']
Topic 2 top words: ['international', 'people', 'public', 'distinct', 'term']
Topic 3 top words: ['white', 'sinking', 'rms', 'ocean', 'ship']
Topic 4 top words: ['percent', 'migration', 'effects', 'countries', 'immigration']
Topic 5 top words: ['international', 'people', 'public', 'distinct', 'term']
Topic 6 top words: ['international', 'people', 'public', 'distinct', 'term']
Topic 7 top words: ['prejudice', 'racist', 'morgan', 'practices', 'social']
Topic 8 top words: ['international', 'people', 'public', 'distinct', 'term']
Topic 9 top words: ['international', 'people', 'public', 'distinct', 'term']


In [ ]:
topic_results = model.transform(vectors)
df['topic'] = topic_results.argmax(axis=1)
df.head()

,title,content,summary,topic
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope...",3
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar...",7
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...,4
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc...",1
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...,7


### NMF (Non-Negative Matrix Factorization)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(df['summary'].values.astype('U'))
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
tfidf = pd.DataFrame(denselist, columns=feature_names)
tfidf.head()

,147,15,17,1837,1912,1913,19th,20th,21,224,...,voting,voyage,wall,wave,western,white,workers,works,world,york
0,0.000000,0.095086,0.000000,0.000000,0.095086,0.000000,0.000000,0.000000,0.000000,0.095086,...,0.000000,0.095086,0.000000,0.000000,0.000000,0.190171,0.000000,0.095086,0.000000,0.095086
1,0.000000,0.000000,0.106841,0.106841,0.000000,0.106841,0.106841,0.106841,0.106841,0.000000,...,0.000000,0.000000,0.213682,0.106841,0.106841,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.087722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087722,0.000000,0.087722,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.098268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
print(tfidf.shape)

(5, 258)


https://arxiv.org/pdf/1706.05084.pdf

Figure 1: Illustration of NMF model for topic modeling

In [ ]:
from sklearn.decomposition import NMF

model = NMF(init='random', n_components=10, random_state=42)
model.fit(vectors)

for index, topic in enumerate(model.components_):
    print(f'Topic {index} top words: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]]}')

Topic 0 top words: ['disaster', 'white', 'rms', 'ocean', 'ship']
Topic 1 top words: ['racist', 'ideology', 'prejudice', 'practices', 'social']
Topic 2 top words: ['percent', 'migration', 'countries', 'effects', 'immigration']
Topic 3 top words: ['operated', 'line', 'star', 'ship', 'ocean']
Topic 4 top words: ['wall', 'street', 'american', 'including', 'morgan']
Topic 5 top words: ['called', 'active', 'right', 'vote', 'suffrage']
Topic 6 top words: ['ideology', 'racist', 'racism', 'social', 'practices']
Topic 7 top words: ['banking', 'head', '31', 'investment', 'sr']
Topic 8 top words: ['york', 'single', 'passenger', 'sank', 'april']
Topic 9 top words: ['passive', 'elections', 'vote', 'right', 'suffrage']


In [ ]:
topic_results = model.transform(vectors)
df['topic'] = topic_results.argmax(axis=1)
df.head()

,title,content,summary,topic
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope...",8
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar...",7
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...,2
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc...",9
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...,1


In [ ]:
suggested_words = []
for i in range(len(df)):
    topic = model.components_[df.loc[i].topic]
    keywords = ' '.join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]])
    suggested_words.extend(wikipedia.search(keywords, results=5))

suggested_words

['Passengers of the Titanic',
 'Lifeboats of the Titanic',
 'Titanic',
 'Sinking of the Titanic',
 'PS General Slocum',
 'Al-Rajhi Bank',
 'J. P. Morgan',
 'Bank of America',
 'Lehman Brothers',
 'Wallenberg family',
 'Immigration',
 'Immigration to the United States',
 'Immigration to Sweden',
 'Immigration to Norway',
 'History of immigration to the United States',
 'Suffrage',
 "Women's suffrage",
 'Universal suffrage',
 'Voting rights in Belgium',
 'Non-citizen suffrage',
 'Racism',
 'Prejudice',
 'Nazism',
 'Woke',
 'Social dominance orientation']

In [ ]:
# https://medium.com/analytics-vidhya/text-summarization-using-spacy-ca4867c6b744
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

nlp = spacy.load('en_core_web_md')

In [ ]:
# doc = nlp(df.loc[0]['content'])
summary_text = ' '.join([txt for txt in df.summary])
# print(summary_text)
doc = nlp(summary_text)
len(list(doc.sents))
keyword = []
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in stopwords or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.text)

# count most frequent words
freq_word = Counter(keyword)
print(freq_word.most_common(5))

# normalize for better processing
max_freq = Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)

print(freq_word.most_common(5))


[('immigration', 4), ('right', 4), ('vote', 4), ('suffrage', 4), ('ship', 3)]
[('immigration', 1.0), ('right', 1.0), ('vote', 1.0), ('suffrage', 1.0), ('ship', 0.75)]


In [ ]:
# weights based on frequency
sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent] += freq_word[word.text]
            else:
                sent_strength[sent] = freq_word[word.text]

print(sent_strength)

{RMS Titanic was a British passenger liner, operated by the White Star Line, that sank in the North Atlantic Ocean on 15 April 1912 after striking an iceberg during her maiden voyage from Southampton, England to New York City, United States.: 8.75, Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making it the deadliest sinking of a single ship up to that time.: 3.75, It remains the deadliest peacetime sinking of an ocean liner or cruise ship.: 3.5, The disaster drew public attention, provided foundational material for the disaster film genre, and has inspired many artistic works.
: 4.0, RMS Titanic was the largest ship afloat at the time she entered service and the second of three Olympic-class ocean liners operated by the White Star Line.: 6.5, John Pierpont Morgan Sr. (April 17, 1837 – March 31, 1913) was an American financier and investment banker who dominated corporate finance on Wall Street throughout the Gilded Age.: 5.75, As the head of the banking firm

In [ ]:
summary = nlargest(10, sent_strength, key=sent_strength.get)
summary = ' '.join([w.text for w in summary])
summary

"RMS Titanic was a British passenger liner, operated by the White Star Line, that sank in the North Atlantic Ocean on 15 April 1912 after striking an iceberg during her maiden voyage from Southampton, England to New York City, United States. Suffrage, political franchise, or simply franchise, is the right to vote in public, political elections and referendums (although the term is sometimes used for any right to vote). In some languages, and occasionally in English, the right to vote is called active suffrage, as distinct from passive suffrage, which is the right to stand for election. Studies show that the elimination of barriers to migration would have profound effects on world GDP, with estimates of gains ranging between 67 and 147 percent for the scenarios in which 37 to 53 percent of the developing countries' workers migrate to the developed countries. RMS Titanic was the largest ship afloat at the time she entered service and the second of three Olympic-class ocean liners operate

# SPARQL-dataframe

* https://pypi.org/project/SPARQLWrapper/
* https://sparqlwrapper.readthedocs.io/en/latest/main.html
* https://pypi.org/project/sparql-dataframe/

In [ ]:
# pip install sparql-dataframe

In [ ]:
import sparql_dataframe

endpoint = "http://dbpedia.org/sparql"

q = """
SELECT ?label ?ship ?owner ?status ?port ?route
WHERE
{
 ?ship dbp:shipName ?label .
 ?ship rdf:type dbo:Ship .
 ?ship dbo:owner ?owner FILTER ( ?owner = dbr:White_Star_Line ) .
 ?ship dbo:status ?status .
 ?ship dbp:shipRegistry ?port .
 ?ship dbp:shipRoute ?route .
}
LIMIT 50
"""

sparql_df = sparql_dataframe.get(endpoint, q)
sparql_df.head()

,label,ship,owner,status,port,route
0,SS Arabic,http://dbpedia.org/resource/SS_Arabic_(1881),http://dbpedia.org/resource/White_Star_Line,Broken up August 1901,http://dbpedia.org/resource/Rotterdam,*Liverpool-New York \n*San Francisco-Hong Kong...
1,SS Arabic,http://dbpedia.org/resource/SS_Arabic_(1881),http://dbpedia.org/resource/White_Star_Line,Sold to theHolland America Linein February 1890,http://dbpedia.org/resource/Rotterdam,*Liverpool-New York \n*San Francisco-Hong Kong...
2,SS Spaarndam,http://dbpedia.org/resource/SS_Arabic_(1881),http://dbpedia.org/resource/White_Star_Line,Broken up August 1901,http://dbpedia.org/resource/Rotterdam,*Liverpool-New York \n*San Francisco-Hong Kong...
3,SS Spaarndam,http://dbpedia.org/resource/SS_Arabic_(1881),http://dbpedia.org/resource/White_Star_Line,Sold to theHolland America Linein February 1890,http://dbpedia.org/resource/Rotterdam,*Liverpool-New York \n*San Francisco-Hong Kong...
4,SS Arabic,http://dbpedia.org/resource/SS_Arabic_(1881),http://dbpedia.org/resource/White_Star_Line,Broken up August 1901,http://dbpedia.org/resource/Liverpool,*Liverpool-New York \n*San Francisco-Hong Kong...


## Knowledge Graph Extension

https://colab.research.google.com/github/om-hb/kgextension/blob/master/examples/book_genre_prediction.ipynb

The kgextension package allows to access and use Linked Open Data to augment existing datasets. It enables to incorporate knowledge graph information in pandas. DataFrames then can be used within the scikit-learn pipeline.

Its functionality includes:

* Linking datasets to any Linked Open Data (LOD) Source such as DBpedia, WikiData or the EU Open Data Portal
* Generation of new features from the LOD Sources
* Hierarchy-based feature selection algorithms
* Data Integration of features from different sources

https://kgextension.readthedocs.io/en/latest/

In [ ]:
# !pip install kgextension

In [ ]:
# get the data
import pandas as pd

titles = pd.read_csv('titles.csv')
titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,48,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7


In [ ]:
from kgextension.linking_sklearn import DbpediaLookupLinker

linker = DbpediaLookupLinker(column='title')
df_enhanced = linker.fit_transform(titles.head())
df_enhanced.head()

DBpedia Lookup Linker: Querying DLL:   0%|          | 0/5 [00:00<?, ?it/s]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,new_link
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,48,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN,http://dbpedia.org/resource/Defender_(associat...
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2,http://dbpedia.org/resource/Taxi_Driver
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8,http://dbpedia.org/resource/Monty_Python_and_t...
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8,http://dbpedia.org/resource/Monty_Python's_Lif...
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7,http://dbpedia.org/resource/The_Exorcist_(film)


In [ ]:
df.head()

,title,content,summary,topic
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope...",8
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar...",7
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...,2
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc...",9
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...,1


In [ ]:
linker = DbpediaLookupLinker(column='title')
df_enhanced2 = linker.fit_transform(df.head())
df_enhanced2.head()

DBpedia Lookup Linker: Querying DLL:   0%|          | 0/5 [00:00<?, ?it/s]

,title,content,summary,topic,new_link
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope...",8,http://dbpedia.org/resource/RMS_Titanic
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar...",7,http://dbpedia.org/resource/J._P._Morgan
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...,2,http://dbpedia.org/resource/Slovenia
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc...",9,http://dbpedia.org/resource/Women's_suffrage
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...,1,http://dbpedia.org/resource/Racism


In [ ]:
# https://kgextension.readthedocs.io/en/latest/source/usage_generators.html#specific-relation-generator
from kgextension.generator_sklearn import SpecificRelationGenerator

generator = SpecificRelationGenerator(columns=['new_link'], direct_relation='http://purl.org/dc/terms/subject')
df_enhanced2 = generator.fit_transform(df_enhanced2)
df_enhanced2.head()

Column:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\cliff\anaconda3\lib\site-packages\kgextension\sparql_helper.py:395: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(query_result)


,title,content,summary,topic,new_link,"new_link_in_boolean_http://dbpedia.org/resource/Category:Businesspeople_from_Hartford,_Connecticut","new_link_in_boolean_http://dbpedia.org/resource/Category:Burials_at_Cedar_Hill_Cemetery_(Hartford,_Connecticut)",new_link_in_boolean_http://dbpedia.org/resource/Category:U.S._Steel_people,new_link_in_boolean_http://dbpedia.org/resource/Category:20th-century_American_businesspeople,new_link_in_boolean_http://dbpedia.org/resource/Category:American_art_collectors,...,new_link_in_boolean_http://dbpedia.org/resource/Category:Countries_in_Europe,new_link_in_boolean_http://dbpedia.org/resource/Category:Member_states_of_NATO,new_link_in_boolean_http://dbpedia.org/resource/Category:Member_states_of_the_Union_for_the_Mediterranean,new_link_in_boolean_http://dbpedia.org/resource/Category:Republics,new_link_in_boolean_http://dbpedia.org/resource/Category:Southern_European_countries,new_link_in_boolean_http://dbpedia.org/resource/Category:Central_European_countries,new_link_in_boolean_http://dbpedia.org/resource/Category:Member_states_of_the_European_Union,new_link_in_boolean_http://dbpedia.org/resource/Category:Member_states_of_the_Three_Seas_Initiative,new_link_in_boolean_http://dbpedia.org/resource/Category:Women's_suffrage,new_link_in_boolean_http://dbpedia.org/resource/Category:Suffrage
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope...",8,http://dbpedia.org/resource/RMS_Titanic,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar...",7,http://dbpedia.org/resource/J._P._Morgan,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...,2,http://dbpedia.org/resource/Slovenia,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc...",9,http://dbpedia.org/resource/Women's_suffrage,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...,1,http://dbpedia.org/resource/Racism,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# https://kgextension.readthedocs.io/en/latest/source/usage_generators.html#data-properties-generator
from kgextension.linking_sklearn import DbpediaLookupLinker
from kgextension.generator import data_properties_generator

linker = DbpediaLookupLinker(column='title')
df_props = linker.fit_transform(df.head())


df_data_properties = data_properties_generator(df_props, 'new_link')
df_data_properties

DBpedia Lookup Linker: Querying DLL:   0%|          | 0/5 [00:00<?, ?it/s]

Column:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\cliff\anaconda3\lib\site-packages\kgextension\sparql_helper.py:395: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(query_result)


,title,content,summary,topic,new_link,new_link_data_http://dbpedia.org/ontology/PopulatedPlace/area,new_link_data_http://dbpedia.org/ontology/PopulatedPlace/populationDensity,new_link_data_http://dbpedia.org/ontology/abstract,new_link_data_http://dbpedia.org/ontology/area,new_link_data_http://dbpedia.org/ontology/birthDate,...,new_link_data_http://dbpedia.org/property/x,new_link_data_http://dbpedia.org/property/y,new_link_data_http://dbpedia.org/property/years,new_link_data_http://www.georss.org/georss/point,new_link_data_http://www.w3.org/2000/01/rdf-schema#comment,new_link_data_http://www.w3.org/2000/01/rdf-schema#label,new_link_data_http://www.w3.org/2003/01/geo/wgs84_pos#geometry,new_link_data_http://www.w3.org/2003/01/geo/wgs84_pos#lat,new_link_data_http://www.w3.org/2003/01/geo/wgs84_pos#long,new_link_data_http://xmlns.com/foaf/0.1/name
0,Titanic,"RMS Titanic was a British passenger liner, ope...","RMS Titanic was a British passenger liner, ope...",8,http://dbpedia.org/resource/RMS_Titanic,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,RMS Titanic,NaN,NaN,NaN,NaN
1,J. P. Morgan,"John Pierpont Morgan Sr. (April 17, 1837 – Mar...","John Pierpont Morgan Sr. (April 17, 1837 – Mar...",7,http://dbpedia.org/resource/J._P._Morgan,NaN,NaN,John Pierpont Morgan (17 April 1837 – 31 Maret...,NaN,1837-04-17,...,NaN,NaN,1904,NaN,John Pierpont Morgan (17 April 1837 – 31 Maret...,J. P. 모건,NaN,NaN,NaN,J. P. Morgan
2,Immigration,Immigration is the international movement of p...,Immigration is the international movement of p...,2,http://dbpedia.org/resource/Slovenia,20271.0,103.0,"La Slovenia, ufficialmente Repubblica di Slove...",2.0271e+10,NaN,...,0.345,0.058,NaN,46.05 14.5,斯洛維尼亞共和国（斯洛維尼亞語：Republika Slovenija），通稱斯洛文尼亚（斯...,Eslovénia,POINT(14.5 46.049999237061),46.1167,14.8167,Republic of Slovenia
3,Suffrage,"Suffrage, political franchise, or simply franc...","Suffrage, political franchise, or simply franc...",9,http://dbpedia.org/resource/Women's_suffrage,NaN,NaN,女性参政権（じょせいさんせいけん）とは、女性が直接または間接的に国や地方自治体の政治に参加す...,NaN,NaN,...,NaN,NaN,NaN,NaN,Con suffragio femminile si indica il diritto d...,Жіноче виборче право,NaN,NaN,NaN,NaN
4,Racism,Racism is discrimination and prejudice towards...,Racism is discrimination and prejudice towards...,1,http://dbpedia.org/resource/Racism,NaN,NaN,العنصرية هي الاعتقاد بأن هناك فروقاً وعناصر مو...,NaN,NaN,...,NaN,NaN,NaN,NaN,Ρατσισμός είναι η αντίληψη ότι οι άνθρωποι δεν...,Расизм,NaN,NaN,NaN,NaN
